<a href="https://colab.research.google.com/github/BosenkoTM/Data-analytics-tools-for-solving-applied-problems/blob/main/marketing_analytics_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Постановка задачи**

Директор по маркетингу сообщил нам, что недавние маркетинговые иссследования оказались не такими эффективными, как ожидалось. Нам нужно проанализировать набор данных, чтобы понять проблему.

Исследовательский анализ данных

* Имеются ли нулевые значения или выбросы? 
* Существуют ли какие-либо переменные, требующие преобразований?
* Есть ли какие-нибудь полезные переменные, которые вы можете спроектировать с заданными данными?

Статистический анализ

* Какие факторы  связаны с количеством покупок в магазине?
* С точки зрения общего объема покупок дела в США значительно лучше, чем в остальном мире?


Визуализация данных

* Какая маркетинговая кампания наиболее успешна?
* Как выглядит средний клиент этой компании?
* Какие продукты работают лучше всего?

# **Load the packages**

In [ ]:
!pip install dython

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
pd.options.mode.chained_assignment = None

# **Load the data**

Для этого проекта использовался [набор маркетинговых данных](https://www.kaggle.com/datasets/lancengck/marketing-data) из задачи  команды iFood Brain в роли аналитиков данных. Этот набор данных содержит социально-демографические и фирмографические характеристики 2240 клиентов.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
# Load dataset
df = pd.read_csv('marketing_data.csv').iloc[: , 1:]

In [ ]:
# Rename Pandas columns to lower case
df.columns = df.columns.str.lower()

In [ ]:
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

In [ ]:
# Examine the data
df.head()

In [ ]:
# Overview of all variables, their datatypes
df.info()

# **Preprocess the data**

In [ ]:
# Clean up column names that contain whitespace
df.columns = df.columns.str.replace(' ', '')

In [ ]:
# Transform income column to a numerical
df['income'] = df['income'].str.replace(',','').str.replace('$','').astype('float')

In [ ]:
import datetime 

In [ ]:
current_year = datetime.date.today().year
current_year

In [ ]:
# Replace 'year_birth' with 'age'
df['age'] = current_year - df['year_birth']

In [ ]:
# Modify date of enrollment to total number of months since enrollment
df['enrollment_month'] = (pd.to_datetime('now') - pd.to_datetime(df['dt_customer'])) // np.timedelta64(1,'M')  

In [ ]:
# Rename the column 'response'
df = df.rename(columns = {'response': 'acceptedcmp6'})

In [ ]:
# Drop unnecessary columns
df = df.drop(['year_birth', 'dt_customer'], axis = 1)

In [ ]:
df.describe()

Мы можем выделить некоторые выбросы как по возрасту, так и по доходу. Похоже, у нас есть клиенты, которым больше 100 лет, или их доход на семью превышает 600 000 долларов США!

In [ ]:
!pip install gitly

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from gitly.colab.plot import GitlyPlotter

In [ ]:
gitly = GitlyPlotter('github')

In [ ]:
fig = make_subplots(rows = 1, cols = 2)

fig.add_trace(go.Box(y = df['age'], name = 'age'), row = 1, col = 1)
fig.add_trace(go.Box(y = df['income'], name = 'income'), row = 1, col = 2)

fig.update_layout(showlegend = False)

gitly.show(fig)

У нас есть 4 клиента, которые являются исключениями. Один из них зарабатывает 666 666 долларов США, а троим из них больше 100 лет!

In [ ]:
# Find outliers
outliers_age = df[df['age'] > 100].index

# Remove outliers
df.drop(outliers_age, inplace = True)

In [ ]:
# Find outlier
outliers_income = df[df['income'] > 200000].index

# Remove outlier
df.drop(outliers_income, inplace = True)

In [ ]:
df['education'].value_counts()

С точки зрения образования, и «2-й цикл», и «магистр» относятся к одному и тому же уровню образования. Это основано на Европейском пространстве высшего образования (EHEA). Поэтому мы объединим два уровня образования под словом «магистр». Кроме того, «выпускной» немного вводит в заблуждение как уровень образования. Мы предположим, что это относится к «бакалавриату» и перефразируем его как таковое.

In [ ]:
# Replace '2n cycle' with 'master'
df['education'] = df['education'].apply(lambda x: 'master' if str(x) == '2n cycle' else str(x))

In [ ]:
# Replace 'graduation' with 'undergraduate'
df['education'] = df['education'].apply(lambda x: 'undergraduate' if str(x) == 'graduation' else str(x))

In [ ]:
df['marital_status'].value_counts()

С точки зрения семейного положения, 'yolo', 'alone', и 'absurd'могут быть истолкованы и приняты как означающие «не замужем», и поэтому эти статусы будут объединены в «холост».

In [ ]:
# Merge 'yolo', 'absurd', and 'alone' under 'single'
df['marital_status'] = df['marital_status'].apply(lambda x: 'single' if str(x) in ['alone', 'yolo', 'absurd'] else str(x))

In [ ]:
df['country'].value_counts()

sp     1094
sa      335
ca      268
aus     160
ind     147
ger     120
us      109
me        3
Name: country, dtype: int64

# **Check for missing values**

In [ ]:
df.isnull().sum()

У нас отсутствуют данные о доходах 24 наших клиентов.

**Определим X и Y**

In [ ]:
X = df.drop('numstorepurchases', axis = 1)

In [ ]:
y = df['numstorepurchases']

**Create test and train data**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Isolate X and y variables, and perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [ ]:
transformer = ColumnTransformer(transformers = [('simple_imputer', SimpleImputer(strategy = 'median'), ['income'])], remainder = 'passthrough')

**Examine collinearity**

Включение функций, которые сильно коррелируют друг с другом или являются мультиколлинеарными, добавляет шум и неточность, поэтому нам нужно попытаться уменьшить это.

Создание тепловой карты корреляции — хороший способ визуализировать потенциальную коллинеарность. Эмпирическое правило состоит в том, что если корреляция между двумя независимыми переменными больше 0,8, тогда будет существовать мультиколлинеарность.

In [ ]:
X_tr = transformer.fit_transform(X_train)

In [ ]:
X_tr = pd.DataFrame(data = X_tr, columns = X.columns)

In [ ]:
from dython.nominal import associations

In [ ]:
complete_correlation = associations(X_tr, figsize = (32, 16))

Отсутствие мультиколлинеарности среди независимых переменных.

# **Какие факторы существенно влияют на количество покупок в магазине?**

Мы будем использовать модель CatBoostRegressor с numstorepurchases в качестве целевой переменной, а затем использовать методы объяснимости машинного обучения, чтобы получить представление о том, какие функции предсказывают количество покупок в магазине.

Мы создаем конвейер, который меняет любые отсутствующие значения, применяет надежный масштаб к функциям, преобразует любые категориальные функции в числовые, а затем подгоняет модель CatBoostRegressor.

In [ ]:
!pip install catboost

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from catboost import CatBoostRegressor

In [ ]:
numeric_columns = list(X_train.select_dtypes(exclude = ['object']).columns.values.tolist())
categorical_columns = list(X_train.select_dtypes(include = ['object']).columns.values.tolist())
numeric_transformer = Pipeline(steps = [('simple_imputer', SimpleImputer(strategy = 'median'))])
categorical_transformer = Pipeline(steps = [('one_hot_encoder', OneHotEncoder(sparse = False, handle_unknown = 'ignore'))])

preprocessor = ColumnTransformer(transformers = [('numeric', numeric_transformer, numeric_columns),
                                                 ('categorical', categorical_transformer, categorical_columns)], remainder = 'passthrough')

bundled_pipeline = Pipeline(steps = [('preprocessor', preprocessor),
                                     ('scaler', RobustScaler()),
                                     ('model', CatBoostRegressor(silent = True, random_state = 42))])

In [ ]:
bundled_pipeline.fit(X_train, y_train)

In [ ]:
y_pred = bundled_pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(y_test, y_pred)

**ELI5**

Из этого пайплайна непросто извлечь важные функции. Однако есть библиотека python, которая делает это очень простым, под названием ELI5.

Давайте используем ELI5 для извлечения важности функций из конвейера.

ELI5 необходимо знать все имена функций, чтобы определить важность функций. Применяя однократное кодирование к категориальным переменным в конвейере, мы вводим ряд новых функций. Поэтому сначала нам нужно извлечь эти имена функций и добавить их к известному списку числовых функций. В приведенном ниже коде для этого используется функция named_steps, встроенная в конвейеры scikit-learn.

In [ ]:
one_hot_columns = list(bundled_pipeline.named_steps['preprocessor'].named_transformers_['categorical'].named_steps['one_hot_encoder'].get_feature_names_out(input_features = categorical_columns))

In [ ]:
numeric_features_list = list(numeric_columns)
numeric_features_list.extend(one_hot_columns)

In [ ]:
!pip install eli5

In [ ]:
import eli5

To extract the feature importances we then simply need to run this line of code.

In [ ]:
eli5.explain_weights(bundled_pipeline.named_steps['model'], top = 50, feature_names = numeric_features_list)

Здесь мы отмечаем, что «mntwines» и «mntmeatproducts» являются наиболее важными функциями.

**Важные замечание**

* Чем точнее модель, тем надежнее рассчитанные значения важности.

* Вычисленные значения важности описывают, насколько важны функции для модели CatBoostRegressor. Это приблизительное представление о том, насколько важны функции в данных.

# **С точки зрения общего объема покупок дела в США значительно лучше, чем в остальном мире?**

In [ ]:
totalpurchases = df[['numdealspurchases', 'numwebpurchases', 'numcatalogpurchases', 'numstorepurchases', 'country']]

In [ ]:
# Calculate the total number of purchases made through different channels
totalpurchases['totalpurchases'] = totalpurchases['numdealspurchases'] + totalpurchases['numwebpurchases'] + totalpurchases['numcatalogpurchases'] + totalpurchases['numstorepurchases']

In [ ]:
average_purchases_per_country = totalpurchases.groupby('country').agg(total_purchases = ('totalpurchases', 'sum'))

In [ ]:
average_purchases_per_country['total_customers'] = totalpurchases['country'].value_counts()

In [ ]:
average_purchases_per_country['purchases_per_customer'] = np.floor(average_purchases_per_country['total_purchases'] / average_purchases_per_country['total_customers'])

In [ ]:
average_purchases_per_country.assign(country = average_purchases_per_country.index.get_level_values('country'))

In [ ]:
average_purchases_per_country.reset_index(inplace = True)

In [ ]:
average_purchases_per_country = average_purchases_per_country.sort_values(by = 'total_purchases', ascending = False)

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = make_subplots(rows = 1, cols = 2, subplot_titles = ('Total purchases by country', 'Average purchases by country'))

fig.add_trace(go.Bar(x = average_purchases_per_country['country'], y = average_purchases_per_country['total_purchases']), row = 1, col = 1)
fig.add_trace(go.Bar(x = average_purchases_per_country['country'], y = average_purchases_per_country['purchases_per_customer']), row = 1, col = 2)

fig['layout']['xaxis']['title'] = 'Country'
fig['layout']['xaxis2']['title'] = 'Country'
fig['layout']['yaxis']['title'] = 'Total purchases'
fig['layout']['yaxis2']['title'] = 'Average purchases'

fig.update_layout(showlegend = False)

gitly.show(fig)

С точки зрения общего количества покупок США, похоже, не занимает лидирующее место. На самом деле он самый низкий (исключая ME из-за всего 3 записей). Однако, если мы посмотрим на покупки, сделанные на человека в стране,
 то США лидируют в чарте.

# **Есть ли существенная связь между географическим регионом и успехом кампании?**

Мы будем использовать критерий хи-квадрат, чтобы определить связь между двумя категориальными переменными, страной и acceptcmp. Начнем с определения нулевой и альтернативной гипотез.

Нулевая гипотеза H0: Две переменные, country и acceptcmp, не зависят друг от друга.

Альтернативная гипотеза H1: две переменные связаны друг с другом.

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
acceptedcmp1 = pd.crosstab(df['country'], df['acceptedcmp1'])

In [ ]:
c, p, dof, expected = chi2_contingency(acceptedcmp1)

In [ ]:
p

0.8736949588868972

In [ ]:
acceptedcmp2 = pd.crosstab(df['country'], df['acceptedcmp2'])

In [ ]:
c, p, dof, expected = chi2_contingency(acceptedcmp2)

In [ ]:
p

In [ ]:
acceptedcmp3 = pd.crosstab(df['country'], df['acceptedcmp3'])

In [ ]:
c, p, dof, expected = chi2_contingency(acceptedcmp3)

In [ ]:
p

In [ ]:
acceptedcmp4 = pd.crosstab(df['country'], df['acceptedcmp4'])

In [ ]:
c, p, dof, expected = chi2_contingency(acceptedcmp4)

In [ ]:
p

In [ ]:
acceptedcmp5 = pd.crosstab(df['country'], df['acceptedcmp5'])

In [ ]:
c, p, dof, expected = chi2_contingency(acceptedcmp5)

In [ ]:
p

In [ ]:
acceptedcmp6 = pd.crosstab(df['country'], df['acceptedcmp6'])

In [ ]:
c, p, dof, expected = chi2_contingency(acceptedcmp6)

In [ ]:
p

Результаты показывают, что между географическими регионами и успехом кампании нет существенной связи, при этом p-значение для всех стран во всех маркетинговых кампаниях превышает 0,05. Это указывает на недостаточность доказательств, чтобы отвергнуть нулевую гипотезу о том, что географические регионы не имеют отношения к успеху маркетинговой кампании.

Мы можем дополнительно проверить это, построив график уровня принятия кампании в разных странах.

In [ ]:
acceptedcmp_by_country = df.groupby('country').agg(acceptedcmp1 = ('acceptedcmp1', 'mean'), 
                                                   acceptedcmp2 = ('acceptedcmp2', 'mean'),
                                                   acceptedcmp3 = ('acceptedcmp3', 'mean'),
                                                   acceptedcmp4 = ('acceptedcmp4', 'mean'),
                                                   acceptedcmp5 = ('acceptedcmp5', 'mean'),
                                                   acceptedcmp6 = ('acceptedcmp6', 'mean')).reset_index()

In [ ]:
acceptedcmp_by_country

In [ ]:
acceptedcmp_by_country = pd.melt(acceptedcmp_by_country.reset_index(), id_vars = 'country', value_vars = ['acceptedcmp1', 'acceptedcmp2', 'acceptedcmp3', 'acceptedcmp4', 'acceptedcmp5', 'acceptedcmp6'])

In [ ]:
fig = px.histogram(acceptedcmp_by_country, x = 'country', y = 'value', color = 'variable', barmode = 'group')

fig.update_layout(title_text = 'Acceptance rate of marketing campaigns across countries', title_x = 0.5)

fig.update_layout(xaxis_title = 'Country')
fig.update_layout(yaxis_title = 'Accepted (%)')

fig.update_layout(legend = {'title_text': ''})

gitly.show(fig)

Из приведенной выше диаграммы видно, что уровень одобрения (%) каждой кампании в разных странах, как правило, довольно низок и довольно одинаков. Таким образом, это имеет смысл и еще раз подтверждает наш вывод о том, что «страна» не является важной характеристикой для прогнозирования успеха кампании.

Обратите внимание, что набор данных содержит только 3 точки данных о клиентах для Мексики, поэтому уровень одобрения кажется высоким (т. Е. Если 1 клиент принимает кампанию, показатель успеха уже будет на уровне 33%).

# **1. Какая маркетинговая кампания наиболее успешна?**

In [ ]:
accepted_cmp = pd.DataFrame(df[['acceptedcmp1', 
                                'acceptedcmp2', 
                                'acceptedcmp3', 
                                'acceptedcmp4', 
                                'acceptedcmp5', 
                                'acceptedcmp6']].mean() * 100, columns = ['accepted_(%)']).sort_values(by = 'accepted_(%)', ascending = False).reset_index()

In [ ]:
accepted_cmp.reset_index(inplace = True)

In [ ]:
accepted_cmp = accepted_cmp.rename(columns = {'index': 'marketing_campaign', 'level_0': 'index'})

In [ ]:
accepted_cmp.set_index('index', inplace = True)

In [ ]:
fig = px.bar(accepted_cmp, x = 'marketing_campaign', y = 'accepted_(%)')

fig.update_layout(title_text = 'Acceptance rates of each marketing campaign', title_x = 0.5)

fig.update_layout(xaxis_title = 'Marketing campaign')
fig.update_layout(yaxis_title = 'Accepted (%)')

gitly.show(fig)

Основываясь на приведенной выше диаграмме, мы можем сделать вывод, что самая последняя кампания является самой успешной.

# **2. Как выглядит средний клиент этой компании?**

**2.1 Categorical features**

**2.1.1 Education**

In [ ]:
education = df.groupby('education').agg(count = ('education', 'count'))

In [ ]:
education.assign(education = education.index.get_level_values('education'))

In [ ]:
education.reset_index(inplace = True)

In [ ]:
fig = px.pie(education, values = 'count', names = 'education')

gitly.show(fig)

В целом, большинство клиентов имеют высшее образование (50,4%).

**2.1.2 Семейный стату**

In [ ]:
marital_status = df.groupby('marital_status').agg(count = ('marital_status', 'count'))

In [ ]:
marital_status.assign(marital_status = marital_status.index.get_level_values('marital_status'))

In [ ]:
marital_status.reset_index(inplace = True)

In [ ]:
fig = px.pie(marital_status, values = 'count', names = 'marital_status')

gitly.show(fig)

Почти 40% клиентов состоят в браке, 25,8% живут вместе, а 21,7% не замужем.

**2.1.3 Country**

In [ ]:
country = df.groupby('country').agg(count = ('country', 'count'))

In [ ]:
country.assign(country = country.index.get_level_values('country'))

In [ ]:
country.reset_index(inplace = True)

In [ ]:
fig = px.pie(country, values = 'count', names = 'country')

gitly.show(fig)

Почти половина клиентов из Испании. Следующим по величине пулом клиентов является ЮАР (Южная Африка) с 15%, затем следует третья СА (Канада) с 12%.

**2.1.4 Dependents**

In [ ]:
dependents = df[['kidhome', 'teenhome']].value_counts().reset_index()

In [ ]:
dependents['index'] = np.arange(1, dependents.shape[0] + 1)

In [ ]:
dependents = dependents.set_index('index')

In [ ]:
dependents['kidhome'] = dependents['kidhome'].astype('string') + 'kid' 

In [ ]:
dependents['kidhome'] = dependents['kidhome'].replace(to_replace = r'(kid)', value = r' \1', regex = True)

In [ ]:
dependents['teenhome'] = dependents['teenhome'].astype('string') + 'teen' 

In [ ]:
dependents['teenhome'] = dependents['teenhome'].replace(to_replace = r'(teen)', value = r' \1', regex = True)

In [ ]:
dependents['dependenthome'] = dependents['kidhome']  + ' & ' + dependents['teenhome']

In [ ]:
dependents = dependents.rename(columns = {0: 'count'})

In [ ]:
fig = px.pie(dependents, values = 'count', names = 'dependenthome')

gitly.show(fig)

* Только у 28,5% клиентов нет детей (хотя бы ребенок или подросток в семье).

* 71,5% клиентов имеют в семье хотя бы 1 ребенка или 1 подростка.

**2.2 Numerical features**

In [ ]:
numerical_features = pd.DataFrame((df[['age', 'income']].mean()))

In [ ]:
numerical_features = numerical_features.rename(columns = {0: 'numerical_feature'})

In [ ]:
numerical_features

Средний покупатель...

* 53 года
* из Испании
* высшее образование
* зарабатывает около 52 000 долларов США
* состоит в отношениях, т.е. состоит в браке или вместе
* имеет по крайней мере ребенка (ребенка или подростка)